# Hydration Free Energy Calculations using GNNImplicit Solvent

This requires https://github.com/fjclark/GNNImplicitSolvent/tree/feature-multiple-molecules.

In [ ]:
!git clone https://github.com/Honza-R/PL-REX.git PL-REX
!cd PL-REX && git sparse-checkout init --cone
!cd PL-REX && git sparse-checkout set 009-CDK2/structures_pl-rex

In [1]:
!ls PL-REX/009-CDK2/structures_pl-rex

3QQK  3QTS  3QTX  3QXP	3R8Z  3RAH  3RJC  3RK9	3RNI  3S00  3SQQ
3QTQ  3QTU  3QTZ  3R8U	3R9D  3RAK  3RK5  3RKB	3RPV  3S0O
3QTR  3QTW  3QU0  3R8V	3R9N  3RAL  3RK7  3RMF	3RPY  3S1H


In [8]:
from pathlib import Path
from rdkit import Chem
from openff.toolkit import ForceField, Molecule, Topology
from Simulation.helper_functions import get_gnn_sim, MODEL_PATH, SOLVENT_DICT
from openmm.app import HBonds
from rdkit import Chem
from openff.toolkit import ForceField, Molecule, Topology
from Simulation.helper_functions import get_gnn_sim, MODEL_PATH, SOLVENT_DICT, create_vac_sim, create_gnn_sim
from openmm.app import HBonds
from openbabel import pybel
from loguru import logger
from openff.units import unit

KJ_TO_KCAL = (1 * unit.kilojoule).to(unit.kilocalorie).magnitude
FORCE_FIELD = "openff_no_water-3.0.0-alpha0.offxml"
PLREX_STRUCTURES = Path("PL-REX/009-CDK2/structures_pl-rex")

## Load and process the receptor

Add topology information to the pdb using obabel

In [3]:
def fix_charges(mol: Chem.Mol) -> None:
    """
    Assign simple formal charges to atoms based on crude valence rules,
    then sanitize the molecule.
    """

    for atom in mol.GetAtoms():
        sym = atom.GetSymbol()

        if sym == "N":
            total_valence = sum(bond.GetBondTypeAsDouble() for bond in atom.GetBonds())
            # Four-coordinate nitrogen should usually be +1
            if total_valence == 4 and atom.GetFormalCharge() == 0:
                atom.SetFormalCharge(1)
                logger.info(f"Set charge +1 on N atom {atom.GetIdx()}")

        elif sym == "O":
            total_valence = sum(bond.GetBondTypeAsDouble() for bond in atom.GetBonds())
            # Single-bonded oxygen often -1 (e.g. deprotonated)
            if total_valence == 1 and atom.GetFormalCharge() == 0:
                atom.SetFormalCharge(-1)
                logger.info(f"Set charge -1 on O atom {atom.GetIdx()}")

    Chem.SanitizeMol(mol)
    logger.info(f"Total molecular charge: {Chem.GetFormalCharge(mol)}")

## Calculate solvation free energies

In [4]:
def make_sim(off_topology, solvent: str, num_confs: int = 1):
    """
    Build a GNN-based simulation object using your helper.
    """
    sim = get_gnn_sim(
        off_topology=off_topology,
        solvent=solvent,
        model_path=MODEL_PATH,
        solvent_dict=SOLVENT_DICT,
        cache=None,
        save_name=None,
        partial_charges=None,
        forcefield=FORCE_FIELD,
        constraints=HBonds,
        num_confs=num_confs,
    )
    return sim

In [5]:
def single_energy(sim):
    """
    Extract total energy (in kJ/mol) from your reference system.
    """
    return sim._ref_system.calculate_energy()._value

In [9]:
def prepare_receptor(system_dir: Path):
    """
    1. Read receptor.pdb via PyBel
    2. Write receptor.sdf
    3. Load SDF with RDKit (no sanitize)
    4. Split into fragments, fix charges, convert to OpenFF Molecule objects
    """
    pdb_path = system_dir / "receptor.pdb"
    sdf_path = system_dir / "receptor.sdf"

    logger.info(f"[{system_dir.name}] Reading receptor from {pdb_path}")

    pdb_structures = list(pybel.readfile("pdb", str(pdb_path)))
    assert len(pdb_structures) == 1, f"Expected 1 PDB structure in {pdb_path}"
    pdb = pdb_structures[0]
    initial_num_atoms = len(pdb.atoms)
    logger.info(f"[{system_dir.name}] Number of atoms in PDB: {initial_num_atoms}")

    # Write to SDF
    pdb.write("sdf", str(sdf_path), overwrite=True)

    # Read back with RDKit (non-sanitized)
    rdkit_mol = Chem.SDMolSupplier(str(sdf_path), sanitize=False, removeHs=False)[0]
    frags = Chem.GetMolFrags(rdkit_mol, asMols=True, sanitizeFrags=False)

    off_frags = []
    for i, frag in enumerate(frags):
        logger.info(f"[{system_dir.name}] Processing fragment {i} with {frag.GetNumAtoms()} atoms")
        fix_charges(frag)
        off_mol = Molecule.from_rdkit(
            frag,
            allow_undefined_stereo=True,
            hydrogens_are_explicit=True,
        )
        off_frags.append(off_mol)

    total_atoms = sum(m.n_atoms for m in off_frags)
    logger.info(f"[{system_dir.name}] Total atoms after conversion: {total_atoms}")
    assert total_atoms == initial_num_atoms, (
        f"Atom count mismatch for {system_dir.name}: "
        f"PDB={initial_num_atoms}, OFF={total_atoms}"
    )

    total_charge = sum(m.total_charge for m in off_frags)
    logger.info(f"[{system_dir.name}] Total receptor charge: {total_charge}")

    return off_frags

In [10]:
def prepare_ligand(system_dir: Path):
    """
    Load ligand.sdf as an OpenFF Molecule.
    """
    lig_path = system_dir / "ligand.sdf"
    logger.info(f"[{system_dir.name}] Reading ligand from {lig_path}")
    ligand = Molecule.from_file(str(lig_path), allow_undefined_stereo=True)
    return ligand


In [11]:
def compute_ddg(system_dir: Path):
    """
    Compute ΔΔG_solv using your GNN-based implicit solvent energy model
    for a single protein–ligand system in `system_dir`.
    """
    name = system_dir.name
    logger.info(f"\n===== Processing system: {name} =====")

    # Prepare receptor and ligand
    receptor_off_mols = prepare_receptor(system_dir)
    ligand = prepare_ligand(system_dir)

    # OpenFF topologies
    receptor_top = Topology.from_molecules(receptor_off_mols)
    ligand_top = Topology.from_molecules([ligand])
    complex_top = Topology.from_molecules(receptor_off_mols + [ligand])

    # ------------------
    # Solvent energies
    # ------------------
    logger.info(f"[{name}] Computing energies in solvent (TIP3P)")
    complex_solv = single_energy(make_sim(complex_top, "tip3p", num_confs=1))
    ligand_solv = single_energy(make_sim(ligand_top, "tip3p", num_confs=1))
    receptor_solv = single_energy(make_sim(receptor_top, "tip3p", num_confs=1))

    # ------------------
    # Vacuum energies
    # ------------------
    logger.info(f"[{name}] Computing energies in vacuum")
    complex_vac = single_energy(make_sim(complex_top, "vac", num_confs=1))
    ligand_vac = single_energy(make_sim(ligand_top, "vac", num_confs=1))
    receptor_vac = single_energy(make_sim(receptor_top, "vac", num_confs=1))

    # ------------------
    # ΔΔG_solv
    # ------------------
    ddG_solv = (complex_solv - complex_vac) \
               - (ligand_solv - ligand_vac) \
               - (receptor_solv - receptor_vac)

    logger.info(f"[{name}] Complex in Solvent: {complex_solv * KJ_TO_KCAL:.3f} kcal/mol")
    logger.info(f"[{name}] Ligand in Solvent:  {ligand_solv * KJ_TO_KCAL:.3f} kcal/mol")
    logger.info(f"[{name}] Receptor in Solvent:{receptor_solv * KJ_TO_KCAL:.3f} kcal/mol")
    logger.info(f"[{name}] Complex in Vacuum:  {complex_vac * KJ_TO_KCAL:.3f} kcal/mol")
    logger.info(f"[{name}] Ligand in Vacuum:   {ligand_vac * KJ_TO_KCAL:.3f} kcal/mol")
    logger.info(f"[{name}] Receptor in Vacuum: {receptor_vac * KJ_TO_KCAL:.3f} kcal/mol")
    logger.info(f"[{name}] ΔΔG_solv = {ddG_solv * KJ_TO_KCAL:.3f} kcal/mol")

    return {
        "system": name,
        "complex_solv_kj": complex_solv,
        "ligand_solv_kj": ligand_solv,
        "receptor_solv_kj": receptor_solv,
        "complex_vac_kj": complex_vac,
        "ligand_vac_kj": ligand_vac,
        "receptor_vac_kj": receptor_vac,
        "ddG_solv_kj": ddG_solv,
        "ddG_solv_kcal": ddG_solv * KJ_TO_KCAL,
    }


In [12]:
if not PLREX_STRUCTURES.exists():
    raise FileNotFoundError(f"Directory not found: {PLREX_STRUCTURES}")

results = []

for system_dir in sorted(PLREX_STRUCTURES.iterdir()):
    if not system_dir.is_dir():
        continue

    pdb_file = system_dir / "receptor.pdb"
    lig_file = system_dir / "ligand.sdf"

    if not (pdb_file.exists() and lig_file.exists()):
        logger.warning(f"Skipping {system_dir.name}: missing receptor.pdb or ligand.sdf")
        continue

    try:
        res = compute_ddg(system_dir)
        results.append(res)
    except Exception as e:
        logger.exception(f"Error processing {system_dir.name}: {e}")


2025-12-11 13:28:32.272 | INFO     | __main__:compute_ddg:7 - 
===== Processing system: 3QQK =====
2025-12-11 13:28:32.273 | INFO     | __main__:prepare_receptor:11 - [3QQK] Reading receptor from PL-REX/009-CDK2/structures_pl-rex/3QQK/receptor.pdb
2025-12-11 13:28:32.299 | INFO     | __main__:prepare_receptor:17 - [3QQK] Number of atoms in PDB: 1593
2025-12-11 13:28:32.402 | INFO     | __main__:prepare_receptor:28 - [3QQK] Processing fragment 0 with 237 atoms
2025-12-11 13:28:32.404 | INFO     | __main__:fix_charges:22 - Set charge -1 on O atom 31
2025-12-11 13:28:32.404 | INFO     | __main__:fix_charges:15 - Set charge +1 on N atom 50
2025-12-11 13:28:32.405 | INFO     | __main__:fix_charges:22 - Set charge -1 on O atom 94
2025-12-11 13:28:32.405 | INFO     | __main__:fix_charges:15 - Set charge +1 on N atom 215
2025-12-11 13:28:32.406 | INFO     | __main__:fix_charges:25 - Total molecular charge: 0
2025-12-11 13:28:32.418 | INFO     | __main__:prepare_receptor:28 - [3QQK] Processing 

Using OpenFF forcefield: openff_no_water-3.0.0-alpha0.offxml
setting charges using default OpenFF method. This is AM1-BCC for force fields < 2.3.0, and ASH-GC for 2.3.0 and above.


[13:28:33] WARNING: Proton(s) added/removed

[13:28:34] WARNING: Proton(s) added/removed

[13:28:36] WARNING: Proton(s) added/removed

[13:28:37] WARNING: Proton(s) added/removed

[13:28:38] WARNING: Proton(s) added/removed

[13:28:39] WARNING: Proton(s) added/removed

[13:28:40] WARNING: Proton(s) added/removed

[13:28:41] WARNING: Proton(s) added/removed



Using OpenFF forcefield: openff_no_water-3.0.0-alpha0.offxml
setting charges using default OpenFF method. This is AM1-BCC for force fields < 2.3.0, and ASH-GC for 2.3.0 and above.
Using OpenFF forcefield: openff_no_water-3.0.0-alpha0.offxml
setting charges using default OpenFF method. This is AM1-BCC for force fields < 2.3.0, and ASH-GC for 2.3.0 and above.
Using OpenFF forcefield: openff_no_water-3.0.0-alpha0.offxml
setting charges using default OpenFF method. This is AM1-BCC for force fields < 2.3.0, and ASH-GC for 2.3.0 and above.
Platform:  GPU  ready
Using OpenFF forcefield: openff_no_water-3.0.0-alpha0.offxml
setting charges using default OpenFF method. This is AM1-BCC for force fields < 2.3.0, and ASH-GC for 2.3.0 and above.
Platform:  GPU  ready
Platform:  GPU  ready
Platform:  GPU  ready
All parallel systems have the same forces as the reference System
Using OpenFF forcefield: openff_no_water-3.0.0-alpha0.offxml
setting charges using default OpenFF method. This is AM1-BCC for 

2025-12-11 13:33:20.907 | INFO     | __main__:compute_ddg:29 - [3QQK] Computing energies in vacuum


...

Using OpenFF forcefield: openff_no_water-3.0.0-alpha0.offxml
setting charges using default OpenFF method. This is AM1-BCC for force fields < 2.3.0, and ASH-GC for 2.3.0 and above.
Using OpenFF forcefield: openff_no_water-3.0.0-alpha0.offxml
setting charges using default OpenFF method. This is AM1-BCC for force fields < 2.3.0, and ASH-GC for 2.3.0 and above.
Using OpenFF forcefield: openff_no_water-3.0.0-alpha0.offxml
setting charges using default OpenFF method. This is AM1-BCC for force fields < 2.3.0, and ASH-GC for 2.3.0 and above.
Using OpenFF forcefield: openff_no_water-3.0.0-alpha0.offxml
setting charges using default OpenFF method. This is AM1-BCC for force fields < 2.3.0, and ASH-GC for 2.3.0 and above.
Platform:  GPU  ready
Using OpenFF forcefield: openff_no_water-3.0.0-alpha0.offxml
setting charges using default OpenFF method. This is AM1-BCC for force fields < 2.3.0, and ASH-GC for 2.3.0 and above.
Platform:  GPU  ready
Platform:  GPU  ready
Platform:  GPU  ready
All parallel

2025-12-11 16:03:51.776 | INFO     | __main__:compute_ddg:29 - [3SQQ] Computing energies in vacuum


Using OpenFF forcefield: openff_no_water-3.0.0-alpha0.offxml
setting charges using default OpenFF method. This is AM1-BCC for force fields < 2.3.0, and ASH-GC for 2.3.0 and above.
Using OpenFF forcefield: openff_no_water-3.0.0-alpha0.offxml
setting charges using default OpenFF method. This is AM1-BCC for force fields < 2.3.0, and ASH-GC for 2.3.0 and above.
Using OpenFF forcefield: openff_no_water-3.0.0-alpha0.offxml
setting charges using default OpenFF method. This is AM1-BCC for force fields < 2.3.0, and ASH-GC for 2.3.0 and above.
Platform:  GPU  ready
Using OpenFF forcefield: openff_no_water-3.0.0-alpha0.offxml
setting charges using default OpenFF method. This is AM1-BCC for force fields < 2.3.0, and ASH-GC for 2.3.0 and above.
Platform:  GPU  ready
Platform:  GPU  ready
Platform:  GPU  ready
All parallel systems have the same forces as the reference System
Using OpenFF forcefield: openff_no_water-3.0.0-alpha0.offxml
setting charges using default OpenFF method. This is AM1-BCC for 

2025-12-11 16:04:15.539 | INFO     | __main__:compute_ddg:41 - [3SQQ] Complex in Solvent: -1990.992 kcal/mol
2025-12-11 16:04:15.540 | INFO     | __main__:compute_ddg:42 - [3SQQ] Ligand in Solvent:  -297.859 kcal/mol
2025-12-11 16:04:15.540 | INFO     | __main__:compute_ddg:43 - [3SQQ] Receptor in Solvent:-1857.201 kcal/mol
2025-12-11 16:04:15.540 | INFO     | __main__:compute_ddg:44 - [3SQQ] Complex in Vacuum:  -813.700 kcal/mol
2025-12-11 16:04:15.540 | INFO     | __main__:compute_ddg:45 - [3SQQ] Ligand in Vacuum:   -260.301 kcal/mol
2025-12-11 16:04:15.540 | INFO     | __main__:compute_ddg:46 - [3SQQ] Receptor in Vacuum: -447.353 kcal/mol
2025-12-11 16:04:15.540 | INFO     | __main__:compute_ddg:47 - [3SQQ] ΔΔG_solv = 270.114 kcal/mol


Platform:  GPU  ready
All parallel systems have the same forces as the reference System


In [19]:
import pandas as pd

df = pd.DataFrame(results)

# ---- Convert kJ → kcal ----
KJ_TO_KCAL = 0.239005736
kj_cols = [col for col in df.columns if col.endswith("_kj")]

for col in kj_cols:
    df[col.replace("_kj", "_kcal")] = df[col] * KJ_TO_KCAL

# ---- Export kcal-only table ----
kcal_cols = ["system"] + [col for col in df.columns if col.endswith("_kcal")]
df[kcal_cols].to_csv("ddg_results_kcal.csv", index=False)

logger.info(f"Finished. Processed {len(results)} systems.")
logger.info("Saved results to ddg_results_kcal.csv")

# ---- Print summary ----
for _, r in df.iterrows():
    print(f"===== {r['system']} =====")
    print(f"Complex Solvent (kcal/mol):   {r['complex_solv_kcal']:.3f}")
    print(f"Ligand Solvent (kcal/mol):    {r['ligand_solv_kcal']:.3f}")
    print(f"Receptor Solvent (kcal/mol):  {r['receptor_solv_kcal']:.3f}")
    print(f"Complex Vacuum (kcal/mol):    {r['complex_vac_kcal']:.3f}")
    print(f"Ligand Vacuum (kcal/mol):     {r['ligand_vac_kcal']:.3f}")
    print(f"Receptor Vacuum (kcal/mol):   {r['receptor_vac_kcal']:.3f}")
    print(f"ΔΔG_solv (kcal/mol):          {r['ddG_solv_kcal']:.3f}\n")


2025-12-11 16:31:47.187 | INFO     | __main__:<module>:16 - Finished. Processed 31 systems.
2025-12-11 16:31:47.188 | INFO     | __main__:<module>:17 - Saved results to ddg_results_kcal.csv


===== 3QQK =====
Complex Solvent (kcal/mol):   -1842.835
Ligand Solvent (kcal/mol):    -159.806
Receptor Solvent (kcal/mol):  -1854.570
Complex Vacuum (kcal/mol):    -656.056
Ligand Vacuum (kcal/mol):     -140.173
Receptor Vacuum (kcal/mol):   -451.785
ΔΔG_solv (kcal/mol):          235.639

===== 3QTQ =====
Complex Solvent (kcal/mol):   -1885.082
Ligand Solvent (kcal/mol):    -209.776
Receptor Solvent (kcal/mol):  -1853.026
Complex Vacuum (kcal/mol):    -714.451
Ligand Vacuum (kcal/mol):     -187.941
Receptor Vacuum (kcal/mol):   -450.525
ΔΔG_solv (kcal/mol):          253.705

===== 3QTR =====
Complex Solvent (kcal/mol):   -1771.926
Ligand Solvent (kcal/mol):    -149.016
Receptor Solvent (kcal/mol):  -1823.589
Complex Vacuum (kcal/mol):    -647.023
Ligand Vacuum (kcal/mol):     -129.031
Receptor Vacuum (kcal/mol):   -449.694
ΔΔG_solv (kcal/mol):          268.977

===== 3QTS =====
Complex Solvent (kcal/mol):   -1848.852
Ligand Solvent (kcal/mol):    -141.275
Receptor Solvent (kcal/mol):